# Competição DSA de Machine Learning - Edição Janeiro/2019
## CRISP-DM: Data Preparation
**Autor:** Wanderson Marques - wdsmarques@gmail.com

Esse Jupyter Notebook contém o **pré-processamento** do conjunto de dados. Considerando a metodologia CRISP-DM, essa atividade refere-se à terceira fase, a preparação dos dados. 

<img src="imgs/dataPreparation.jpg" />

### Carregar bibliotecas

In [6]:
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import seaborn as sns
from pylab import rcParams

In [7]:
# Exibir gráficos dentro do Jupyter Notebook
% matplotlib inline

# Definir tamanho padrão para os gráficos
rcParams['figure.figsize'] = 17, 4

### Carregar dataset

In [8]:
dataset = pd.read_csv('datasets/dataset_treino.csv')

# Descartar identificador dos pacientes
dataset.drop(['id'], axis=1, inplace=True)

dataset.head()

,num_gestacoes,glicose,pressao_sanguinea,grossura_pele,insulina,bmi,indice_historico,idade,classe
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


### Separar conjuntos de treino (70%) e validação (30%)

O conjunto de validação será utilizado posteriormente, para verificar a capacidade preditiva do modelo. Ele é isolado para que não receba nenhum tipo de viés.

In [9]:
train, valid = train_test_split(dataset.copy(), test_size=0.3, random_state=1)
dataset = None

### Gravar conjuntos de validação

In [10]:
valid.to_csv('datasets/valid.csv', index=False)

### Tratar valores nulos

São tratamentos comuns para valores nulos:
- Exclusão do atributo (caso ele seja nulo para grande parte das instâncias)
- Exclusão da instância (caso ela seja nula para grande parte dos atributos)
- Imputação por estatísticas simples, como média, mediana ou moda (podem ser calculadas para sub-amostras)
- Imputação por regressão e modelos preditivos

In [11]:
# Converter atributos zerados (fora de domínio) para nulo
naoZero = ['glicose', 'pressao_sanguinea', 'grossura_pele', 'insulina', 'bmi']
for coluna in naoZero:
    train.loc[train[coluna] == 0, coluna] = np.NaN

In [12]:
train.isnull().sum()

num_gestacoes          0
glicose                4
pressao_sanguinea     21
grossura_pele        125
insulina             205
bmi                    6
indice_historico       0
idade                  0
classe                 0
dtype: int64

##### Imputar em glicose, pressao_sanguinea e bmi usando a mediana dos grupos (diabéticos e não diabéticos)

In [13]:
mediana_nao_diabeticos = np.median((train.loc[train['classe'] == 0, 'glicose']).dropna())
mediana_diabeticos = np.median((train.loc[train['classe'] == 1, 'glicose']).dropna())

train.loc[(train['glicose'].isnull()) & (train['classe'] == 0), 'glicose'] = mediana_nao_diabeticos
train.loc[(train['glicose'].isnull()) & (train['classe'] == 1), 'glicose'] = mediana_diabeticos

print("A mediana para diabéticos e não diabéticos é, respectivamente, ", mediana_nao_diabeticos, mediana_diabeticos)

A mediana para diabéticos e não diabéticos é, respectivamente,  106.0 141.5


In [14]:
mediana_nao_diabeticos = np.median((train.loc[train['classe'] == 0, 'bmi']).dropna())
mediana_diabeticos = np.median((train.loc[train['classe'] == 1, 'bmi']).dropna())

train.loc[(train['bmi'].isnull()) & (train['classe'] == 0), 'bmi'] = mediana_nao_diabeticos
train.loc[(train['bmi'].isnull()) & (train['classe'] == 1), 'bmi'] = mediana_diabeticos

print("A mediana para diabéticos e não diabéticos é, respectivamente, ", mediana_nao_diabeticos, mediana_diabeticos)

A mediana para diabéticos e não diabéticos é, respectivamente,  29.8 34.3


In [15]:
mediana_nao_diabeticos = np.median((train.loc[train['classe'] == 0, 'pressao_sanguinea']).dropna())
mediana_diabeticos = np.median((train.loc[train['classe'] == 1, 'pressao_sanguinea']).dropna())

train.loc[(train['pressao_sanguinea'].isnull()) & (train['classe'] == 0), 'pressao_sanguinea'] = mediana_nao_diabeticos
train.loc[(train['pressao_sanguinea'].isnull()) & (train['classe'] == 1), 'pressao_sanguinea'] = mediana_diabeticos

print("A mediana para diabéticos e não diabéticos é, respectivamente, ", mediana_nao_diabeticos, mediana_diabeticos)

A mediana para diabéticos e não diabéticos é, respectivamente,  70.0 74.0


##### Imputar em grossura da pele e insulina utilizando regressão linear simples

In [16]:
# Função para estimar um valor y dado um valor x com Regressão Linear
def LinearReg(name, df, x_label, y_label):
    filterdf = (df[y_label].isnull()) & (df[x_label].notnull())
    dropped =  df.dropna()
    x = dropped[[x_label]]
    y = dropped[[y_label]]
    
    model = LinearRegression()
    model.fit(x, y)
    
    if len(df.loc[filterdf, [x_label]]) > 0:
        df.loc[filterdf, [y_label]] = model.predict(df.loc[filterdf, [x_label]])
    
    joblib.dump(model, filename='models/'+name+'.pkl')
    
    return df

In [17]:
# Imputar grossura_pele utilizando bmi
train = LinearReg('reg-grossura_pele', train, 'bmi', 'grossura_pele')

# Imputar insulina utilizando glicose
train = LinearReg('reg-insulina', train, 'glicose', 'insulina')

In [18]:
train.isnull().sum()

num_gestacoes        0
glicose              0
pressao_sanguinea    0
grossura_pele        0
insulina             0
bmi                  0
indice_historico     0
idade                0
classe               0
dtype: int64

### Tratar outliers

São tratamentos comuns para valores nulos:

- Exclusão da instância
- Correção por estatísticas simples ou modelos preditivos

Não será feito neste momento nenhum tratamento de outliers

### Balancear classes

Em problemas de classificação onde as classes (0 e 1 para classe diabetes) estão desbalanceadas é comum que seja feito o balanceamento para evitar um modelo enviesado. Esse balanceamento pode ser feito limitando a quantidade de instâncias pela classe de menor representatividade ou utilizando técnicas de sobre-amostragem (oversampling). 

In [19]:
# Classes desbalanceadas
train['classe'].value_counts()

0    279
1    141
Name: classe, dtype: int64

In [20]:
# Separar dados em variáveis X (preditoras) e y (predita)
X = train.drop(['classe'], axis=1)
y = train['classe']

In [21]:
# Realizar oversampling
sm = SMOTE()
X_res, y_res = sm.fit_sample(X, y)

y = pd.DataFrame(y_res, columns=['classe'])

In [22]:
# Classes balanceadas
y['classe'].value_counts()

1    279
0    279
Name: classe, dtype: int64

### Padronizar atributos

É comum que cada atributo possua uma escala diferente, porém isso pode ser um problema para a maioria dos modelos preditivos. Portanto, é necessário transformar os dados, colocando todos os atributos em uma mesma escala.

In [23]:
# Preparar o scaler
# Outros scalers: RobustScaler, MinMaxScaler
scaler = StandardScaler()
scaler.fit(X)

# Salvar o scaler para uso futuro
joblib.dump(scaler, filename='models/scaler.pkl')

['models/scaler.pkl']

In [24]:
# Transformar os dados
X = pd.DataFrame(scaler.transform(X_res), columns=train.drop(['classe'], axis=1).columns)

train = pd.concat([X, y], axis=1)

### Gravar conjuntos de treino pré-processado

Após a finalização do pré-processamento, os dados transformados são salvos.

In [25]:
train.head()

,num_gestacoes,glicose,pressao_sanguinea,grossura_pele,insulina,bmi,indice_historico,idade,classe
0,0.081683,-0.958528,-0.128467,0.375059,-0.646852,0.130852,-0.087298,-0.915004,0
1,0.081683,-0.828195,-0.615292,-0.066020,-0.602656,-0.025319,-0.942702,-0.173586,1
2,0.385285,1.126797,1.007455,1.570262,3.912298,0.925903,0.431947,0.073553,0
3,-1.132724,-0.893362,2.305653,1.072261,-0.806473,1.593177,1.638517,0.155933,0
4,0.992489,-0.372030,0.682906,0.275459,-0.270717,0.528377,1.956667,0.814972,1


In [26]:
train.to_csv('datasets/train-preprocessado.csv', index=False)